In [1]:
import pandas as pd 
import numpy as np
import sklearn
import os
import gc
# parallel apply!
import swifter
from tqdm.autonotebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
data_dir = './data'
age_test_df = pd.read_csv(os.path.join(data_dir, "age_test.csv"), header=None)
age_train_df = pd.read_csv(os.path.join(data_dir, "age_train.csv"), header=None)
user_app_actived_df = pd.read_csv(os.path.join(data_dir, "user_app_actived.csv"), header=None)

In [3]:
age_train_df.columns = ['uid', 'age_group']
age_test_df.columns = ['uid']

user_app_actived_df.columns = ['uid', 'appId']

In [4]:
user_app_actived_df['appId'] = user_app_actived_df['appId'].progress_apply(lambda s: s.replace('#', ' '))

In [5]:
all_apps = set()
for apps in tqdm(user_app_actived_df['appId'], leave=False):
    all_apps.update(apps.split(' '))

In [6]:
import sklearn.feature_extraction.text
# w2v_vectorizer = sklearn.feature_extraction.text.CountVectorizer()
# w2v_app_vectors = w2v_vectorizer.fit_transform(user_app_actived_df['app_category'])
tf_idf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(vocabulary=all_apps)
tf_idf_vectors = tf_idf_vectorizer.fit_transform(user_app_actived_df['appId'])

In [7]:
indexes = pd.DataFrame(user_app_actived_df['uid'])
indexes['index'] = [i for i in range(indexes.shape[0])]

In [8]:
train_indexes = age_train_df.merge(indexes, on='uid', how='left') 
test_indexes = age_test_df.merge(indexes, on='uid', how='left') 

In [9]:
train_tfidf = tf_idf_vectors[list(train_indexes['index'].values), :]
test_tfidf = tf_idf_vectors[list(test_indexes['index'].values), :]

In [17]:
np.save('train-data/train_tfidf', train_tfidf)
np.save('train-data/test_tfidf', test_tfidf)

In [18]:
# np.load('train-data/tfidf.npy')